In [ ]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Get to the folder we are at
FOLDERNAME = 'Colab\ Notebooks/SC201L18'
%cd drive/MyDrive/$FOLDERNAME

In [ ]:
%ls

In [ ]:
import torch
torch.manual_seed(2)  # Seed for same output

In [ ]:
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')
print('Device:', device)

In [ ]:
import cv2
import numpy as np
import os
IMG_SIZE = 64

def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
  cap = cv2.VideoCapture(path)
  frames = []
  can_use = False
  frame_count = 0
  try:
    while True:
      success, frame = cap.read()
      if not success:
        break
      frame = cv2.resize(frame, resize)
      frames.append(frame)
      frame_count += 1
      if len(frames) == max_frames:
        can_use = True
        break
  finally:
    cap.release()
  return can_use, np.array(frames)/255

In [ ]:
train_path = 'train/'
val_path = 'test/'
folders = os.listdir(train_path)

# Indexing for ture labels
index_to_folder = {}
for t, folder in enumerate(folders):
  index_to_folder[folder] = t

# Getting train data
train_data = []
train_label = []
max_frames = 16
for folder in folders:
  real_folder_path = os.path.join(train_path, folder)
  for video_name in os.listdir(real_folder_path):
    real_video_path = os.path.join(real_folder_path, video_name)
    can_use, video = load_video(real_video_path, max_frames)
    if can_use:
      train_data.append(video)
      train_label.append(index_to_folder[folder])

In [ ]:
print('Number of data:', len(train_data))
print('Data.shape:', train_data[0].shape)

In [ ]:
# Getting val data
val_data = []
val_label = []
for folder in folders:
  real_folder_path = os.path.join(val_path, folder)
  for video_name in os.listdir(real_folder_path):
    real_video_path = os.path.join(real_folder_path, video_name)
    can_use, video = load_video(real_video_path, max_frames)
    if can_use:
      val_data.append(video)
      val_label.append(index_to_folder[folder])

In [ ]:
num_val = len(val_label)
print(num_val)

In [ ]:
import random
random.seed(1)
train_shuffle_list = list(i for i in range(len(train_label)))
random.shuffle(train_shuffle_list)
val_shuffle_list = list(i for i in range(len(val_label)))
random.shuffle(val_shuffle_list)

new_train_data = []
new_train_label = []
for i in train_shuffle_list:
  new_train_data.append(train_data[i])
  new_train_label.append(train_label[i])
# TODO: What did the error message tell us to do here?
new_train_data = torch.tensor(new_train_data, dtype=torch.float32)
new_train_label = torch.tensor(new_train_label)

new_val_data = []
new_val_label = []
for i in val_shuffle_list:
  new_val_data.append(val_data[i])
  new_val_label.append(val_label[i])
# TODO: What did the error message tell us to do here?
new_val_data = torch.tensor(new_val_data, dtype=torch.float32)
new_val_label = torch.tensor(new_val_label)

In [ ]:
from torch.utils.data import DataLoader
batch_size = 8
mini_train_data = DataLoader(new_train_data, batch_size=batch_size)
mini_train_label = DataLoader(new_train_label, batch_size=batch_size)
mini_val_data = DataLoader(new_val_data, batch_size=batch_size)
mini_val_label = DataLoader(new_val_label, batch_size=batch_size)

In [ ]:
print(next(iter(mini_train_data)).shape)

In [ ]:
from torch.nn.modules.batchnorm import BatchNorm2d
import torch.nn as nn
class CNNLSTM(nn.Module):
  def __init__(self, embedding_dim, hidden_dim, output_dim):
    super().__init__()
    #####################################
    #                                   #
    #               TODO:               #
    #                                   #
    #####################################

  def forward(self, x):
    # 8 x 16 x 64 x 64 x 3
    # Get info from each F (Frame) -> Convolute each F
    
    # Get 1024 (embedding_dim) neurons from each frame
    
    # Pile up all embedding neurons from each frame to a new tensor
    
    # Feed into LSTM and return 
    pass
    

In [ ]:
embedding_dim = 1024
hidden_dim = 256
print_every = 100
output_dim = len(index_to_folder)

In [ ]:
model = CNNLSTM(embedding_dim, hidden_dim, output_dim)
model = model.cuda()

In [ ]:
# Training Procedure
def training(num_epoch, model, mini_trains, mini_train_labels, mini_vals, mini_val_labels, device, loss_function, optimizer):
  for epoch in range(num_epoch):
    num_iters = 0
    train_label_iter = iter(mini_train_labels)
    for x in mini_trains:
      y = next(train_label_iter)
      model.train()
      x = x.to(device)
      y = y.to(device)
      scores = model(x)  
      loss = loss_function(scores, y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      if num_iters % 20 == 0:
        evaluate_predictor(model, epoch, mini_vals, mini_val_labels, device, num_iters)
      num_iters += 1

In [ ]:
# Validating Procedure
def evaluate_predictor(model, epoch, mini_vals, mini_val_labels, device, num_data):
  model.eval()
  val_label_iter = iter(mini_val_labels)
  with torch.no_grad():
    acc_count = 0
    for x in mini_vals:
      y = next(val_label_iter)
      x = x.to(device)
      y = y.to(device)
      scores = model(x)
      predictions = scores.max(1)[1]
      acc = predictions.eq(y).sum().item()
      acc_count += acc
    print(f'Epoch[{epoch+1}] Acc: {acc_count/num_val}')

In [ ]:
import torch.optim as optim
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.9, nesterov=True)

In [ ]:
training(7, model, mini_train_data, mini_train_label, mini_val_data, mini_val_label, device, loss_function, optimizer)